In [1]:
import pandas as pd
from constants import DATA_PATH, EOS_FILE, SENTINEL_FILE

sentinel = pd.read_csv(DATA_PATH / SENTINEL_FILE)
eos = pd.read_csv(DATA_PATH / EOS_FILE)

In [2]:
target_column = 'SM1 (%)'
target_value = 50.0

df_majority = eos[eos[target_column] == target_value]

df_minority = eos[eos[target_column] != target_value]

target_fraction = 0.3

new_majority_size = int(len(df_majority) * target_fraction)

df_majority_undersampled = df_majority.sample(
    n=new_majority_size, 
    random_state=42  # 'random_state' ensures the sampling is repeatable
)

eos = pd.concat([df_majority_undersampled, df_minority])

In [3]:
from constants import X_cols_eos, X_cols_sentinel, y_col

X_sentinel = sentinel[X_cols_sentinel].values
X_eos = eos[X_cols_eos].values

y_sentinel = sentinel[y_col].values
y_eos = eos[y_col].values

In [4]:
import tensorflow as tf

2025-11-02 02:08:58.441458: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from model_experiments import PredictionIntervalEstimation

tf.keras.backend.clear_session()

models = {
    "16, 1": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
]),
    "8, 1": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1)
]),
    "2, 1": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(2, activation='relu'),
    tf.keras.layers.Dense(1)
]),
    "4, 1": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1)
]),
    "16, Dropout, 8, Dropout": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.09),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.09),
    tf.keras.layers.Dense(1)
]),
    "16, Dropout": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])
}

eos_results = {}

for param_string, model in models.items():
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    exp = PredictionIntervalEstimation(X_eos, y_eos, satellite="EOS-04")
    results = exp.run_experiment(model, model_param_string=param_string, optimizer=optimizer, epochs=1000)
    eos_results[param_string] = results

Epochs:   1%|▏         | 14/1000 [00:03<04:07,  3.98epoch/s, loss=0.4784, val_loss=0.4928]


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
16, 1: {
    "val": {
        "PICP": 0.9634703196347032,
        "MPIW": 47.19537353515625
    },
    "test": {
        "PICP": 0.9634703196347032,
        "MPIW": 47.139892578125
    }
}


Epochs:   2%|▏         | 16/1000 [00:03<04:01,  4.07epoch/s, loss=0.4795, val_loss=0.4916]

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8, 1: {
    "val": {
        "PICP": 0.954337899543379,
        "MPIW": 46.81431198120117
    },
    "test": {
        "PICP": 0.9497716894977168,
        "MPIW": 46.75381851196289
    }
}


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
16, Dropout, 8, Dropout: {
    "val": {
        "PICP": 0.9497716894977168,
        "MPIW": 51.07823944091797
    },
    "test": {
        "PICP": 0.958904109589041,
        "MPIW": 51.66743087768555
    }
}


Epochs:   2%|▏         | 15/1000 [00:03<04:15,  3.86epoch/s, loss=0.4825, val_loss=0.4935]

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
16, Dropout: {
    "val": {
        "PICP": 0.954337899543379,
        "MPIW": 50.81865310668945
    },
    "test": {
        "PICP": 0.9680365296803652,
        "MPIW": 51.275577545166016
    }
}


In [8]:
from constants import OUTPUT_PATH
import json

with open(OUTPUT_PATH / "pi_estimation" / "EOS-04_metrics.json", "w") as f:
    json.dump(eos_results, f, indent=4)

In [9]:
sentinel_results = {}

for param_string, model in models.items():
    tf.keras.backend.clear_session()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    exp = PredictionIntervalEstimation(X_sentinel, y_sentinel, satellite="Sentinel-1")
    results = exp.run_experiment(model, model_param_string=param_string, optimizer=optimizer, epochs=1000)
    sentinel_results[param_string] = results

Epochs:   5%|▍         | 49/1000 [00:09<03:11,  4.96epoch/s, loss=0.5717, val_loss=0.5361]


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
16, 1: {
    "val": {
        "PICP": 0.967032967032967,
        "MPIW": 45.975364685058594
    },
    "test": {
        "PICP": 0.967032967032967,
        "MPIW": 45.87718200683594
    }
}


Epochs:   5%|▍         | 47/1000 [00:09<03:09,  5.02epoch/s, loss=0.5711, val_loss=0.5365]


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8, 1: {
    "val": {
        "PICP": 0.9285714285714286,
        "MPIW": 45.5745849609375
    },
    "test": {
        "PICP": 0.9230769230769231,
        "MPIW": 45.490970611572266
    }
}


Epochs:  25%|██▍       | 249/1000 [00:45<02:18,  5.43epoch/s, loss=0.5785, val_loss=0.5471]


...

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4, 1: {
    "val": {
        "PICP": 0.9560439560439561,
        "MPIW": 45.750247955322266
    },
    "test": {
        "PICP": 0.9560439560439561,
        "MPIW": 45.68120193481445
    }
}


Epochs:   2%|▏         | 16/1000 [00:02<02:49,  5.82epoch/s, loss=0.5782, val_loss=0.5428]


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
16, Dropout, 8, Dropout: {
    "val": {
        "PICP": 0.967032967032967,
        "MPIW": 53.97932434082031
    },
    "test": {
        "PICP": 0.9505494505494505,
        "MPIW": 54.95872497558594
    }
}


Epochs:   3%|▎         | 28/1000 [00:06<03:31,  4.60epoch/s, loss=0.5769, val_loss=0.5385]


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
16, Dropout: {
    "val": {
        "PICP": 0.967032967032967,
        "MPIW": 53.199745178222656
    },
    "test": {
        "PICP": 0.945054945054945,
        "MPIW": 54.10997772216797
    }
}
